In [ ]:
!git clone https://github.com/ARBML/klaam
%cd klaam

Cloning into 'klaam'...
remote: Enumerating objects: 759, done.
remote: Counting objects: 100% (584/584), done.
remote: Compressing objects: 100% (291/291), done.
remote: Total 759 (delta 276), reused 517 (delta 257), pack-reused 175
Receiving objects: 100% (759/759), 136.69 MiB | 32.92 MiB/s, done.
Resolving deltas: 100% (298/298), done.
Checking out files: 100% (175/175), done.
/content/klaam


In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install pyquran

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 334 kB 20.1 MB/s 


In [ ]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize
import numpy
from klaam.utils.audio import get_audio
from klaam import SpeechRecognition
import pyquran as q

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
 from klaam import SpeechRecognition
 model = SpeechRecognition()

Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Downloading:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/158 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/138 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
suras_names = [None, 'الفاتحة', 'البقرة', 'آل عمران', 'النساء', 'المائدة', 'الأنعام', 'الأعراف', 'الأنفال',
               'التوبة',
               'يونس', 'هود',
               'يوسف', 'الرعد', 'ابراهيم', 'الحجر', 'النحل', 'الإسراء', 'الكهف', 'مريم', 'طه', 'الأنبياء',
               'الحج',
               'المؤمنون',
               'النور', 'الفرقان', 'الشعراء', 'النمل', 'القصص', 'العنكبوت', 'الروم', 'لقمان', 'السجدة',
               'الأحزاب',
               'سبإ',
               'فاطر', 'يس', 'الصافات', 'ص', 'الزمر', 'غافر', 'فصلت', 'الشورى', 'الزخرف', 'الدخان',
               'الجاثية',
               'الأحقاف',
               'محمد', 'الفتح', 'الحجرات', 'ق', 'الذاريات', 'الطور', 'النجم', 'القمر', 'الرحمن', 'الواقعة',
               'الحديد',
               'المجادلة', 'الحشر', 'الممتحنة', 'الصف', 'الجمعة', 'المنافقون', 'التغابن', 'الطلاق',
               'التحريم',
               'الملك',
               'القلم', 'الحاقة', 'المعارج', 'نوح', 'الجن', 'المزمل', 'المدثر', 'القيامة', 'الانسان',
               'المرسلات',
               'النبإ',
               'النازعات', 'عبس', 'التكوير', 'الإنفطار', 'المطففين', 'الإنشقاق', 'البروج', 'الطارق',
               'الأعلى',
               'الغاشية',
               'الفجر', 'البلد', 'الشمس', 'الليل', 'الضحى', 'الشرح', 'التين', 'العلق', 'القدر', 'البينة',
               'الزلزلة',
               'العاديات', 'القارعة', 'التكاثر', 'العصر', 'الهمزة', 'الفيل', 'قريش', 'الماعون', 'الكوثر',
               'الكافرون', 'النصر',
               'المسد', 'الإخلاص', 'الفلق', 'الناس']

In [ ]:
def get_sim(aya,recorded_aya):
  toknized_trg = word_tokenize(aya)
  toknized_src = word_tokenize(recorded_aya)
  print(toknized_trg)
  print(toknized_src)
  psrc = 0
  ptrg = 0
  itre = len(toknized_trg)
  trget = []
  while (psrc < itre and ptrg < itre):
    if (psrc == len(toknized_src)):
      trget.append("Missed")
      ptrg += 1
    elif(levenshteinDistanceDP(toknized_src[psrc] , toknized_trg[ptrg]) == 0):
      trget.append("Same")
      psrc += 1
      ptrg += 1
    else:
      diff = levenshteinDistanceDP(toknized_src[psrc] , toknized_trg[ptrg])
      if(len(toknized_trg[ptrg]) > len(toknized_src[psrc]) and diff <= len(toknized_trg[ptrg])/2):
        trget.append("Modefied")
        psrc += 1
        ptrg += 1
      elif(len(toknized_trg[ptrg]) <= len(toknized_src[psrc]) and diff <= len(toknized_src[psrc])/2): 
        trget.append("Modefied")
        psrc += 1
        ptrg += 1
      else:
        if(psrc != len(toknized_src)-1):
          diff = levenshteinDistanceDP(toknized_src[psrc+1] , toknized_trg[ptrg])
        if (diff != 0):
          trget.append("Missed")
          psrc += 1
          ptrg += 1
        else:
          trget.append("Same")
          psrc += 1
          ptrg += 1
          

  print()
  print(trget)

def levenshteinDistanceDP(token1, token2):
    distances = numpy.zeros((len(token1) + 1, len(token2) + 1))
    
    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1
    
    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2
        
    
    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1
    return distances[len(token1)][len(token2)]


In [ ]:
def record_aya():
  input("Press enter to record ")
  get_audio()
  x = model.transcribe('demo.wav')
  print(x)
  return x

In [ ]:
def record():
  recorded_aya = record_aya()
  while(True):
    if(len(word_tokenize(recorded_aya)) > len(word_tokenize(aya))):
      print("Record again")
      recorded_aya = record_aya()
    else:
      break
  return recorded_aya

In [ ]:
get_sura_name = input("Enter sura name : ")
sura_number = suras_names.index(get_sura_name)
sura = q.quran.get_sura(sura_number, with_tashkeel=False, basmalah=False)
print()
for a in range(len(sura)):
  print(a+1 ,"-",  sura[a])

print()

get_aya = input("Choose aya number : ")

aya = q.quran.get_verse(sura_number,int(get_aya))
print(aya)


recorded_aya = record()
get_sim(aya,recorded_aya)

Enter sura name : الفاتحة

1 - بسم الله الرحمن الرحيم
2 - الحمد لله رب العلمين
3 - الرحمن الرحيم
4 - ملك يوم الدين
5 - إياك نعبد وإياك نستعين
6 - اهدنا الصرط المستقيم
7 - صرط الذين أنعمت عليهم غير المغضوب عليهم ولا الضالين

Choose aya number : 1
بسم الله الرحمن الرحيم
Press enter to record 


بس الله وأحمان
['بسم', 'الله', 'الرحمن', 'الرحيم']
['بس', 'الله', 'وأحمان']

['Modefied', 'Same', 'Missed', 'Missed']
